# Traffic prediction with Pytorch

### Librairies

In [2]:
# Helper librairies
import os
import numpy as np
import pandas as pd
import random as rd
from datetime import datetime, date

# Torch
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.utils.data

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns # prettier plots

### Get the train data

In [3]:
data_path = os.getcwd() + "/"

train=pd.read_csv(data_path + "Radar_Traffic_Counts.csv")

print(train.tail())

                                           location_name  location_latitude  \
4603856                         LAMAR BLVD / MANCHACA RD          30.243875   
4603857                         LAMAR BLVD / MANCHACA RD          30.243875   
4603858                         LAMAR BLVD / MANCHACA RD          30.243875   
4603859                  CAPITAL OF TEXAS HWY / CEDAR ST          30.339468   
4603860   LAMAR BLVD / SANDRA MURAIDA WAY (Lamar Bridge)          30.266800   

         location_longitude  Year  Month  Day  Day of Week  Hour  Minute  \
4603856          -97.781705  2019      7   27            6     2      30   
4603857          -97.781705  2019      5   29            3    22       0   
4603858          -97.781705  2019      8    6            2     2      30   
4603859          -97.803558  2019      9   26            4    10       0   
4603860          -97.756051  2019      7    1            1     8       0   

        Time Bin Direction  Volume  
4603856    02:30        NB     